## Automated Extraction of Leaf Area from Photos

##### Take photos of leaves with any camera*, populate a folder, and quickly leaf extract data

*on white background of known dimensions

To answer some important plant biology, horticultural or agronomic research questions it can be important to accurately measure the area of many leaves. This labor intensive process has previously limited the size of datasets used in publications, and a faster solution would vastly improve the size and quality of these datasets.

The following code represents a fast, reliable and simple framework for rapidly extracting leaf area from large numbers of images captured with a camera.

In [1]:
# import packages
import os
import cv2
import skimage
import pandas as pd
import numpy as np
import imutils
from imutils.perspective import four_point_transform
from imutils import contours
from skimage import io
from PIL import Image

In [2]:
# define functions
def getListOfFiles(dirName):
    # create a list of file and sub directories names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

### Set some variables

Edit the folder name and paper size to appropriate values.

Set paper or white background size using any units you prefer.

In [3]:
# set folder name (including path from this jupyter notebook)
folder = "photos"

# set paper or white background size
paper_size = 0.06032246 # use units of m^2

# build list of image files (do not edit)
file_list = getListOfFiles(folder)
if 'photos/.DS_Store' in file_list:
    file_list.remove('photos/.DS_Store')

### Loop over all the photos in folder

Extract leaf area, then add date of capture and leaf area to a dataframe

In [11]:
# reset some variables
z = 0
df = pd.DataFrame()

# loop over all images in the list of image files
while z < len(file_list):

    # load the image
    image = cv2.imread(file_list[z])

    # compute ratio of old height to the new height, clone it and resize it
    ratio = image.shape[0] / 1000.0 # save in case we want to rescale to full size later
    orig = image.copy()
    image = imutils.resize(image, height = 1000)

    # convert resized image to grayscale, blur it slightly, then find edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    
    # apply Otsu's thresholding method to create a binarized version, too
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    # find contours in the edge map, keeping only the largest ones
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True) # approximate the contour
        if len(approx) == 4: # if approx contour has four points, then assume that we found paper
            screenCnt = approx
            break
            
    # apply the four point transform to obtain a top-down view of the paper area only         
    warped = four_point_transform(thresh, screenCnt.reshape(4, 2))  

    # extract leaf area
    n_white = cv2.countNonZero(warped)
    height, width = warped.shape
    n_total = height * width
    leaf_area = (n_white / n_total) * paper_size

    # extract creation time from image metadata
    creation_time = Image.open(imageName)._getexif()[36867]

    if z == 0:
            
        # initialize list of lists
        data = [[creation_time[:10], leaf_area]]

        # Create the pandas DataFrame from list of lists
        df = pd.DataFrame(data, columns=['Date', 'Leaf Area'])
        
        z += 1

    else:
        # append onto existing results
        df2 = {'Date': creation_time[:10],'Leaf Area': leaf_area}
        df = df.append(df2, ignore_index = True)

        z += 1

/var/folders/9_/4gxvqdb51mdbx_cn1phg60bw0000gn/T/ipykernel_41491/3223403259.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
/var/folders/9_/4gxvqdb51mdbx_cn1phg60bw0000gn/T/ipykernel_41491/3223403259.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
/var/folders/9_/4gxvqdb51mdbx_cn1phg60bw0000gn/T/ipykernel_41491/3223403259.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index = True)
/var/folders/9_/4gxvqdb51mdbx_cn1phg60bw0000gn/T/ipykernel_41491/3223403259.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

,Date,Leaf Area
0,2023:05:25,0.001919
1,2023:05:25,0.000868
2,2023:05:25,0.002919
3,2023:05:25,0.003128
4,2023:05:25,0.001746
5,2023:05:25,0.003269
6,2023:05:25,0.000953
7,2023:05:25,0.002317
8,2023:05:25,0.002071
9,2023:05:25,0.002216


### Print the dataframe with image datetime and leaf area

In [12]:
display(df)

,Date,Leaf Area
0,2023:05:25,0.001919
1,2023:05:25,0.000868
2,2023:05:25,0.002919
3,2023:05:25,0.003128
4,2023:05:25,0.001746
5,2023:05:25,0.003269
6,2023:05:25,0.000953
7,2023:05:25,0.002317
8,2023:05:25,0.002071
9,2023:05:25,0.002216


**Authors:**
Matthew Jenkins, 
Dylan Lenczewski-Jowers, 
Fabiola Chavez Lamas